I am going to put this into a SQL database, I am just more comfortable with manipulating data using SQL

In [1]:
import pandas as pd
import sqlite3 as sl

In [2]:
#create database object
conn = sl.connect('covid.db')
c = conn.cursor()

here are the column names that need to be created in our table
'id', 
'sex', 
'patient_type', 
'entry_date', 
'date_symptoms', 
'date_died',
'intubed', 
'pneumonia', 
'age', 
'pregnancy', 
'diabetes', 
'copd',
'asthma', 
'inmsupr', 
'hypertension', 
'other_disease', 
'cardiovascular',
'obesity', 
'renal_chronic', 
'tobacco', 
'contact_other_covid',
'covid_res', 
'icu'

In [9]:
#create table and populate
covid = pd.read_csv('covid.csv', index_col=0, encoding='latin-1')

c.execute('DROP TABLE covid')
c.execute('CREATE TABLE covid (id int, sex int, patient_type int, entry_date datetime, date_symptoms datetime, date_died datetime, intubed int, pneumonia int, age int, pregnancy int, diabetes int, copd int, asthma int,inmsupr int, hypertension int, other_disease int, cardiovascular int, obesity int, renal_chronic int, tobacco int, contact_other_covid int, covid_res int, icu int)')
conn.commit()

covid.to_sql('covid',conn, if_exists='replace')

In [10]:
df = pd.read_sql(''' select * from covid limit 10;''',conn)
df

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,04-05-2020,02-05-2020,9999-99-99,97,2,27,97,...,2,2,2,2,2,2,2,2,1,97
1,1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,...,2,2,2,2,2,2,2,99,1,97
2,167386,1,2,06-04-2020,01-04-2020,9999-99-99,2,2,54,2,...,2,2,2,2,1,2,2,99,1,2
3,0b5948,2,2,17-04-2020,10-04-2020,9999-99-99,2,1,30,97,...,2,2,2,2,2,2,2,99,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,...,2,1,2,1,2,2,2,99,1,2
5,1beec8,2,2,16-04-2020,16-04-2020,29-04-2020,2,1,47,97,...,2,2,2,2,2,2,2,99,1,1
6,1.75E+56,2,2,22-04-2020,13-04-2020,9999-99-99,2,2,63,97,...,2,1,2,2,2,2,2,99,1,2
7,0ce1f5,1,1,21-05-2020,18-05-2020,9999-99-99,97,2,56,2,...,2,1,2,2,2,1,1,1,1,97
8,12b57a,1,1,22-04-2020,18-04-2020,9999-99-99,97,2,41,2,...,2,2,2,2,2,2,2,99,1,97
9,006b91,1,2,23-04-2020,18-04-2020,9999-99-99,1,1,39,2,...,2,2,2,2,1,2,2,99,1,2


In [18]:
# creating some new columns -- days from symptom onset to hospital admission and days from hospital admission to death
df = pd.read_sql('''SELECT id, sex, age, patient_type, date_symptoms, entry_date as admit_date, 
                   CAST((JulianDay(entry_date) - JulianDay(date_symptoms)) as int) as onset_to_admit_days,
                   date_died, JulianDay(date_died) - JulianDay(entry_date) as admit_to_death_days,
                   icu, pneumonia, intubed, pregnancy, diabetes, copd, 
                   asthma, inmsupr, hypertension, other_disease, cardiovascular, obesity, 
                   renal_chronic, tobacco, contact_other_covid, covid_res
                   FROM covid;''',conn)

df

,id,sex,age,patient_type,date_symptoms,admit_date,onset_to_admit_days,date_died,admit_to_death_days,icu,...,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res
0,16169f,2,27,1,02-05-2020,04-05-2020,None,9999-99-99,None,97,...,2,2,2,2,2,2,2,2,2,1
1,1009bf,2,24,1,17-03-2020,19-03-2020,None,9999-99-99,None,97,...,2,2,2,2,2,2,2,2,99,1
2,167386,1,54,2,01-04-2020,06-04-2020,None,9999-99-99,None,2,...,2,2,2,2,2,1,2,2,99,1
3,0b5948,2,30,2,10-04-2020,17-04-2020,None,9999-99-99,None,2,...,2,2,2,2,2,2,2,2,99,1
4,0d01b5,1,60,2,13-04-2020,13-04-2020,None,22-04-2020,None,2,...,2,2,1,2,1,2,2,2,99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566597,01ff60,2,58,1,03-05-2020,13-05-2020,None,9999-99-99,None,97,...,2,2,2,2,2,2,2,2,2,3
566598,047cd1,1,48,1,06-04-2020,07-04-2020,None,9999-99-99,None,97,...,2,2,2,2,2,2,2,2,2,3
566599,1beb81,1,49,2,01-05-2020,14-05-2020,None,9999-99-99,None,2,...,2,2,2,2,2,1,2,2,99,3
566600,16fb02,1,43,1,29-05-2020,31-05-2020,None,9999-99-99,None,97,...,2,2,2,2,2,2,2,2,2,3


In [17]:
df.onset_to_admit_days.value_counts

<bound method IndexOpsMixin.value_counts of 0         None
1         None
2         None
3         None
4         None
          ... 
566597    None
566598    None
566599    None
566600    None
566601    None
Name: onset_to_admit_days, Length: 566602, dtype: object>